In [27]:
import json
import os
import networkx as nx
import geopandas as gpd
from shapely.geometry import LineString

In [2]:
# caricamento file di linee e punti
with open(os.path.join('tmp','graph_network.geojson')) as f:
    grafo_data = json.load(f)

with open(os.path.join('tmp','graph_nodes.geojson')) as f:
    punti_data = json.load(f)

In [3]:
linee_connessione = []

# Estrai le linee di connessione
for feature in grafo_data['features']:
    geometry = feature['geometry']
    coordinates = geometry['coordinates']
    linee_connessione.append(coordinates)

In [11]:
# Crea un nuovo grafo
grafo = nx.Graph()

# Aggiungi le linee di connessione come archi nel grafo
for linea_connessione in linee_connessione:
    punto_a = tuple(linea_connessione[0])
    punto_b = tuple(linea_connessione[1])
    grafo.add_edge(punto_a, punto_b)

In [99]:
punto_partenza = None
punto_arrivo = None

for feature in punti_data['features']:
    properties = feature['properties']
    
    if properties['FARM_CODE'] == 'F03': 
        print(tuple(feature['geometry']['coordinates']))
        punto_partenza = tuple(feature['geometry']['coordinates'])
    elif properties['FARM_CODE'] == 'F14': 
        print(tuple(feature['geometry']['coordinates']))
        punto_arrivo = tuple(feature['geometry']['coordinates'])
        
# Verifica se i punti di partenza e di arrivo sono stati trovati
if punto_partenza is None or punto_arrivo is None:
    print("Punto di partenza o punto di arrivo non trovati.")


(1047128.608926032, 5042617.065792236)
(1066057.220122893, 5028279.2324866485)


In [100]:
# Calcola il percorso più breve utilizzando l'algoritmo di Dijkstra senza pesi sugli archi
percorso = nx.dijkstra_path(grafo, punto_partenza, punto_arrivo)
percorso

[(1047128.608926032, 5042617.065792236),
 (1043466.9210496588, 5040343.504269967),
 (1061008.5513613047, 5036892.4520490775),
 (1062277.6739161336, 5033152.581058687),
 (1066057.220122893, 5028279.2324866485)]

In [101]:
linea = LineString(percorso)
gdf = gpd.GeoDataFrame(geometry=[linea])

crs = 'EPSG:3857'  # Sostituisci con il CRS desiderato
gdf = gdf.set_crs(crs)


In [102]:
gdf.to_file(os.path.join('tmp','shortest.geojson'), driver='GeoJSON')